In [1]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np


# # SKlearn classification models
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.linear_model import RidgeClassifier

#cross validation
from scipy import sparse
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB


/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str},encoding='utf-8')
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str},encoding='utf-8')
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str},encoding='utf-8')

/usr/local/lib/python2.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
voice_test = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str},encoding='utf-8')
sms_test = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str},encoding='utf-8')
wa_test = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str},encoding='utf-8')

In [4]:
voice_test_a = pd.read_csv('../data/voice_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str},encoding='utf-8')
sms_test_a = pd.read_csv('../data/sms_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str},encoding='utf-8')
wa_test_a = pd.read_csv('../data/wa_test_a.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str},encoding='utf-8')

In [5]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})

voice = pd.concat([voice_train,voice_test_a,voice_test],axis=0)
sms = pd.concat([sms_train,sms_test_a,sms_test],axis=0)
wa = pd.concat([wa_train,wa_test_a,wa_test],axis=0)


In [110]:
uid_test = pd.DataFrame({'uid':pd.unique(wa_test['uid'])})
uid_test.to_csv('../data/uid_test_b.csv',index=None)

In [6]:
voice.drop_duplicates(inplace=True)
sms.drop_duplicates(inplace=True)
wa.drop_duplicates(inplace=True)

In [7]:
def single_groupby(data,groupby_col,count_col,methods,prefix,fill=None):
    temp = data.groupby([groupby_col])[count_col].agg(methods).add_prefix(prefix).reset_index()
    if fill !=None:
        temp.fillna(fill,inplace=True)
    return temp



# 通话记录

##### 通话记录不同记录数 通话记录数

In [8]:

voice_opp_num = single_groupby(voice,'uid','opp_num',{'unique_count': lambda x: len(pd.unique(x)),'count':'count'},'voice_opp_num_',0)
voice_opp_head = single_groupby(voice,'uid','opp_head',{'unique_count': lambda x: len(pd.unique(x))},'voice_opp_head_',0)

voice_opp_len=voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)
voice_opp_len_opp_num_unique=voice.groupby(['uid','opp_len'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_opp_len_opp_num_unique_').reset_index().fillna(0)



voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)
voice_call_type_opp_num_unique=voice.groupby(['uid','call_type'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_call_type_opp_num_unique_').reset_index().fillna(0)

voice_in_out_opp_num_unique = voice.groupby(['uid','in_out'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_in_out_opp_num_unique_').reset_index().fillna(0)


In [9]:
voice_each_opp_head_count=voice.groupby(['uid','opp_head'])['uid'].count().unstack().add_prefix('voice_each_opp_head_count_').reset_index().fillna(0)

In [10]:
voice_opp_len_opp_head_unique=voice.groupby(['uid','opp_len'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('voice_opp_len_opp_head_unique_').reset_index().fillna(0)


In [12]:

opp_num_list = voice.groupby(['opp_num'])['uid'].count().sort_values(ascending=False).reset_index()['opp_num'][0:1000].values
voice_each_opp_num_count=voice[voice.opp_num.map(lambda x: x in opp_num_list)].groupby(['uid','opp_num'])['uid'].count().unstack().add_prefix('voice_each_opp_num_count_').reset_index().fillna(0)

In [13]:
def time_gap(start,end):
    if pd.isnull(start):
        return np.nan
    end_day = int(str(end)[0:2])
    start_day = int(str(start)[0:2])
    day_gap = (end_day-start_day)*86400
    
    
    end_hour = int(str(end)[2:4])
    start_hour = int(str(start)[2:4])
    hour_gap = (end_hour-start_hour)*3600
    
    
    
    end_min = int(str(end)[4:6])
    start_min = int(str(start)[4:6])
    min_gap = (end_min-start_min)*60
    
    
    end_sec = int(str(end)[6:8])
    start_sec = int(str(start)[6:8])
    sec_gap = (end_sec-start_sec)
    
    
    return day_gap+hour_gap+min_gap+sec_gap
    

In [14]:
## 通话时长统计
voice['gap_time']=voice[['start_time','end_time']].apply(lambda x: time_gap(x[0],x[1]),axis=1)

voice_gap_time=voice.groupby(['uid'])['gap_time'].agg(['std','max','min','median','mean','sum',np.ptp]).add_prefix('voice_gap_time_').reset_index()





In [15]:
###  通话时间间隔统计
voice_sort = (voice.sort_values(by=['start_time','end_time'],ascending=True)).reset_index()
voice_sort['last_end_time']=voice_sort.groupby(['uid'])['end_time'].apply(lambda i:i.shift(1))
voice_sort['last_gap_time'] = voice_sort[['last_end_time','start_time']].apply(lambda x: time_gap(x[0],x[1]),axis=1)


voice_last_gap_time=voice_sort.groupby(['uid'])['last_gap_time'].agg(['std','max','min','median','mean','sum',np.ptp]).add_prefix('voice_last_gap_time_').reset_index()

In [16]:
voice['start_day']=voice.start_time.map(lambda x: x[0:2])
voice['end_day']=voice.end_time.map(lambda x: x[0:2])

In [17]:
voice_start_day_count = voice.groupby(['uid','start_day'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().fillna(0).add_prefix('voice_start_day_count_').reset_index()

### sms 

In [18]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count'}).add_prefix('sms_opp_num_').reset_index().fillna(0)
sms_opp_head=sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index().fillna(0)
sms_opp_len=sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

sms_opp_len_opp_head_unique=sms.groupby(['uid','opp_len'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('sms_opp_len_opp_head_unique_').reset_index().fillna(0)


sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)


sms_in_out['sms_in_out_0_rate'] = sms_in_out['sms_in_out_0'] / sms_opp_num['sms_opp_num_count']

sms_in_out['sms_in_out_1_rate'] = sms_in_out['sms_in_out_1'] / sms_opp_num['sms_opp_num_count']





In [19]:
# for col in sms_opp_len.columns:
#     if col !='uid':
#         sms_opp_len[col+'_rate'] = sms_opp_len[col] /  sms_opp_num['sms_opp_num_count']

In [20]:
sms['hour'] = sms.start_time.map(lambda x: x[2:4])
sms['day'] = sms.start_time.map(lambda x: x[0:2])

In [21]:
opp_len = [5,7,8,9,10,11,12,13,14]
sms_opp_count = []
for l in opp_len:
    temp = sms[sms.opp_len==l].groupby(['uid','hour'])['uid'].count().unstack().add_prefix('sms_hour_count_opp_len_'+str(l)+'_').reset_index().fillna(0)
    sms_opp_count.append(temp)
    
for l in opp_len:
    temp = sms[sms.opp_len==l].groupby(['uid','day'])['uid'].count().unstack().add_prefix('sms_day_count_opp_len_'+str(l)+'_').reset_index().fillna(0)
    sms_opp_count.append(temp)

In [22]:
sms_each_opp_head_count=sms.groupby(['uid','opp_head'])['uid'].count().unstack().add_prefix('sms_each_opp_head_count_').reset_index().fillna(0)

In [23]:

sms_opp_num_list = sms.groupby(['opp_num'])['uid'].count().sort_values(ascending=False).reset_index()['opp_num'][0:1000].values
sms_each_opp_num_count=sms[sms.opp_num.map(lambda x: x in sms_opp_num_list)].groupby(['uid','opp_num'])['uid'].count().unstack().add_prefix('sms_each_opp_num_count_').reset_index().fillna(0)


#sms_each_opp_num_count=sms.groupby(['uid','opp_num'])['uid'].count().unstack().add_prefix('sms_each_opp_num_count_').reset_index().fillna(0)

In [24]:
###  短信间隔统计
sms_sort = sms.sort_values(by=['uid','start_time'],ascending='True').reset_index()
sms_sort['last_start_time']=sms_sort.groupby(['uid'])['start_time'].apply(lambda i:i.shift(1))
sms_sort['last_start_gap_time'] = sms_sort[['last_start_time','start_time']].apply(lambda x: time_gap(x[0],x[1]),axis=1)
sms_last_start_gap_time=sms_sort.groupby(['uid'])['last_start_gap_time'].agg(['std','max','min','median','mean','sum',np.ptp]).add_prefix('sms_last_start_gap_time_').reset_index()



In [25]:
sms_each_day_count = sms.groupby(['uid','day'])['opp_num'].count().unstack().fillna(0).add_prefix('sms_each_day_count_').reset_index()

In [26]:
sms_each_day_unique_count_opp_head = sms.groupby(['uid','day'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().fillna(0).add_prefix('sms_each_day_unique_count_opp_head_').reset_index()

In [27]:
sms_each_day_unique_count_opp_num = sms.groupby(['uid','day'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().fillna(0).add_prefix('sms_each_day_unique_count_opp_num_').reset_index()

In [28]:
sms_each_day_in_out_0_count = sms[sms.in_out==0].groupby(['uid','day'])['uid'].count().unstack().fillna(0).add_prefix('sms_each_day_in_out_0_count_').reset_index()

In [29]:
sms_each_day_in_out_1_count = sms[sms.in_out==1].groupby(['uid','day'])['uid'].count().unstack().fillna(0).add_prefix('sms_each_day_in_out_1_count_').reset_index()

In [30]:
sms_each_hour_unique_count_opp_head = sms.groupby(['uid','hour'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().fillna(0).add_prefix('sms_each_hour_unique_count_opp_head_').reset_index()

In [31]:
sms_each_hour_unique_count_opp_num = sms.groupby(['uid','hour'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().fillna(0).add_prefix('sms_each_hour_unique_count_opp_num_').reset_index()

In [32]:




sms_in_out_opp_num_unique = sms.groupby(['uid','in_out'])['opp_num'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('sms_in_out_opp_num_unique_').reset_index().fillna(0)
sms_in_out_opp_head_unique = sms.groupby(['uid','in_out'])['opp_head'].agg(lambda x: len(pd.unique(x))).unstack().add_prefix('sms_in_out_opp_head_unique_').reset_index().fillna(0)


## 网站/app记录

In [35]:
wa_a = pd.concat([wa_train,wa_test_a],axis=0)

wa_name_list = wa.groupby(['wa_name'])['uid'].count().sort_values(ascending=False).reset_index()['wa_name'][0:1000].values
wa_each_name_count=wa[wa.wa_name.map(lambda x: x in wa_name_list)].groupby(['uid','wa_name'])['uid'].count().unstack().add_prefix('wa_each_name_count_').reset_index().fillna(0)


##  特征

In [102]:
voice_feature = [voice_opp_num,  ## 统计用户 记录数，用户 不同opp_num 记录数
                 voice_opp_head, ## 统计用户不同 opp_head记录 数
           voice_opp_len,  ## 统计用户 不同 opp_len 的记录数
           voice_call_type, ## 统计用户不同 call_tyoe 记录数
           voice_in_out_opp_num_unique, ## 统计用户不同  in_out 下 不同 opp_num记录数
           voice_opp_len_opp_num_unique, ## 统计不同 opp_len 下 不同 opp_head 记录数
            voice_opp_len_opp_head_unique, ## 统计不同 opp_len 下不同opp_head 的记录数
            voice_call_type_opp_num_unique,  ## 统计不同call_type 下 不同opp_num 的记录数
          voice_gap_time,  ## 通话时长统计量
          voice_last_gap_time, ## 两次通话间隔统计量
        voice_each_opp_num_count, ## 对opp_num one-hot 统计记录数
                 

                ]


sms_feature = [sms_opp_num,  ## 统计用户sms 不同opp_num记录数
              sms_opp_head, ## 统计用户sms 不同opp_head 记录数
              sms_opp_len, ##统计用户 不同opp_len记录数
              sms_opp_len_opp_head_unique, ##统计用户不同opp_len 中 不同opp_head的记录数
              sms_in_out, ## 不同in_out 记录数
              sms_last_start_gap_time, ## 两次短信间隔统计量
               sms_each_day_count, ##每天的短信记录数
               sms_each_day_unique_count_opp_head, ## 每天不同的opp_head 记录数
               
                sms_in_out_opp_head_unique, ## 不同in_out 下 不同opp_head 数

                sms_each_opp_head_count, ## opp_head one-hot 统计记录数
    
               
               
               

               
              ]
#sms_feature =sms_feature+sms_opp_count

wa_feature = [

    
    wa_each_name_count, ##top 1000 wa_name 分组统计记录数

]




In [103]:
train_feature = uid_train
test_feature = uid_test

for feat in voice_feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
for feat in voice_feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
    
for feat in sms_feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
for feat in sms_feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
    
for feat in wa_feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
for feat in wa_feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
    
for feat in voice_sms_feature:
    train_feature=pd.merge(train_feature,feat,how='left',on='uid')
for feat in voice_sms_feature:
    test_feature=pd.merge(test_feature,feat,how='left',on='uid')
    
# train_feature=pd.merge(train_feature,stacking_feat[stacking_col],how='left',on='uid')
# test_feature=pd.merge(test_feature,stacking_feat[stacking_col],how='left',on='uid')


In [104]:
train_feature.to_csv('../data/train_featureV0.csv',index=None,encoding='utf-8')
test_feature.to_csv('../data/test_featureV0.csv',index=None,encoding='utf-8')

In [105]:
print train_feature.shape

(4999, 2315)
